# Imports

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime as dt
from pathlib import Path
from pytz import timezone
from hbt_tools import utils_hbt as hu
from hbt_tools import plots as hp

# Functions

In [ ]:
def filter_ts(ts, date_from, date_to):
    if not date_to:
        ts = ts[(ts.index>=date_from)]
    elif not date_from:
        ts = ts[(ts.index<date_to)]
    else:
        ts = ts[(ts.index>=date_from) & (ts.index<date_to)]

    return ts

# Parameters

In [ ]:
save_dir = Path(r"Q:\Projekte\8000-\8500er\8596\8596.35-Hagnau"
    r"\05 Berechnungen Grundlagen\Datenauswertung\Plots")

mes_dict = {
    'Zulauf Splittfilterbecken': {
        'id': 	1078,
        'units': 'cm',
        'yaxis': 'y1',
    },

    'Trübung Zulauf': {
        'id': 	1077,
        'units': 'NTU',
        'yaxis': 'y2',
    },

    'Trübung Ablauf': {
        'id': 	1081,
        'units': 'NTU',
        'yaxis': 'y2',
    },

    'Niveau Retentionssickerbecken': {
        'id': 	1082,
        'units': 'cm',
        'yaxis': 'y3',
    }
}

# Read Rain

In [ ]:

path_rain = Path(r"Q:\Projekte\8000-\8500er\8596\8596.35-Hagnau"
    r"\05 Berechnungen Grundlagen\Regendaten\therwil_2207-2307.csv")
date_from = dt(2022, 7, 29, 8, tzinfo=timezone('Europe/Zurich'))

rain = hu.read_raindata_agrometeo(path_rain, resolution='10min')
rain = filter_ts(rain, date_from=date_from, date_to=None)
rain['kumulativ'] = rain.cumsum()

# Read Niveau Splittfilter

In [ ]:
path_niveau_filter = Path(r"Q:\Projekte\8000-\8500er\8596\8596.35-Hagnau"
    r"\05 Berechnungen Grundlagen\Datenauswertung"
    r"\Daten-SABA_1min_220701-230731.csv")

date_parser = lambda x: pd.to_datetime(x, format=date_format)
niveau_filter = pd.read_csv(path_niveau_filter, sep=';')
niveau_filter.insert(0, 'Datum', 0)
niveau_filter.iloc[:,3] = niveau_filter.iloc[:,3].str.replace(':', '')
niveau_filter['Datum'] = (niveau_filter.iloc[:,1] + niveau_filter.iloc[:,2]
    + niveau_filter.iloc[:,3])
niveau_filter.drop(columns=niveau_filter.columns[1:4], inplace=True)
niveau_filter.set_index(niveau_filter.columns[0], inplace=True)
date_format = '%d.%m.%Y%H:%M:%S%z'
niveau_filter.index = pd.to_datetime(niveau_filter.index,
    format=date_format)
niveau_filter = niveau_filter * 100 # m -> cm

# Read Data Messdatenbank

In [ ]:
for k, v in mes_dict.items():
    data = hu.hbtdb_get_data(v['id'])
    mes_dict[k].update({'data': data})

# Overview Plot

In [ ]:
fig = go.Figure()

for k, v in mes_dict.items():
    if v['yaxis']:
        yaxis = v['yaxis']
    
    else:
        yaxis = None

    fig.add_trace(go.Scatter(
        x=v['data'].index,
        y=v['data'].iloc[:,0],
        name=k,
        yaxis=yaxis
    ));

# Add rain to plot
fig.add_trace(go.Scatter(
    x=rain.index,
    y=rain.iloc[:,0],
    name='Regen',
    yaxis='y4',
    line=dict(color='blue')
));

# Add level RFB to plot
fig.add_trace(go.Scatter(
    x=niveau_filter.index,
    y=niveau_filter.iloc[:,0],
    name='Niveau über RFB',
    yaxis='y3',
));

fig.add_trace(go.Scatter(
    x=niveau_filter.index,
    y=niveau_filter.iloc[:,1],
    name='Niveau im RFB',
    yaxis='y3',
));

In [ ]:
ylabels = ['Durchfluss [l/s]', 'Trübung [NTU]', 'Niveau [cm]', 'Regen [mm]']

# Create axis objects
props_dict = {}
props_dict.update({
    'xaxis': {
        'domain': [0.1 + 0.01, 1.0]
    }
})

for i, ylabel in enumerate(ylabels):
    yaxis = f'yaxis{i+1}'
    if i==0:
        props_dict.update({yaxis: {}})
        props_dict[yaxis].update({
            'title': ylabel,
            'side': 'left',
            'anchor':'x'
        })

    else:
        props_dict.update({yaxis: {}})
        props_dict[yaxis].update({
            'title': ylabel,
            'side': 'left',
            'overlaying': 'y',
            'autoshift': True,
            'anchor':'free'
        })

props_dict['yaxis1'].update({'anchor': 'x'})

In [ ]:
fig.update_layout(props_dict);

In [ ]:
save_path = save_dir / 'Übersicht-Messungen.html'
fig.write_html(save_path)

# Zulauf

In [ ]:
id = 1078
q_zu = hu.hbtdb_get_data(id)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=q_zu.index,
    y=q_zu.iloc[:,0],
    name='Zulauf Splittfilterbecken',
    mode='lines+markers',
    line=dict(color='grey', width=1),
    marker=dict(color='cyan', size=4)
));

fig.add_trace(go.Scatter(
    x=rain.index,
    y=rain.iloc[:,1],
    name='Regen kumulativ',
    yaxis='y2',
    line=dict(dash='dash', color='blue')
));

In [ ]:
fig.update_layout(
    xaxis={
        'domain': [0.1, 1]
    },
    yaxis1={
        'title': 'Durchfluss [l/s]',
        'anchor':'x'
    },
    yaxis2={
        'title': 'Regen [mm]',
        'overlaying': 'y',
        'anchor':'free',
        'autoshift': True,
    }
);

In [ ]:
fig.update_layout(
    title='Zulauf Splittfilterbecken', font=dict(size=25)
)

fig

In [ ]:
save_path = save_dir / 'Zulauf-Splittfilterbecken.html'
fig.write_html(save_path)

# Niveau RFB

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=niveau_filter.index,
    y=niveau_filter.iloc[:,0],
    name='Niveau Filter',
));

fig.add_trace(go.Scatter(
    x=niveau_filter.index,
    y=niveau_filter.iloc[:,1],
    name='Niveau Splittfilter',
));

In [ ]:
save_path = save_dir / 'Niveau_RFB.html'
fig.write_html(save_path)

# Niveau RSB